In [1]:
pip install pyspark

In [32]:
# Contoh membuat DataFrame sederhana dan operasi dasar
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('HandsOnPertemuan3').getOrCreate()
data = [('James', 'Sales', 3000),
        ('Michael', 'Sales', 4600),
        ('Robert', 'Sales', 4100),
        ('Maria', 'Finance', 3000),]
columns = ['EmployeeName', 'Department', 'Salary']

df = spark.createDataFrame(data, columns)
df.show()

+------------+----------+------+
|EmployeeName|Department|Salary|
+------------+----------+------+
|       James|     Sales|  3000|
|     Michael|     Sales|  4600|
|      Robert|     Sales|  4100|
|       Maria|   Finance|  3000|
+------------+----------+------+



In [24]:
# Contoh operasi transformasi DataFrame
df.select('EmployeeName', 'Salary').show()
df.filter(df.Salary > 3000).show()
df.groupBy('Department').avg('Salary').show()

+------------+------+
|EmployeeName|Salary|
+------------+------+
|       James|  3000|
|     Michael|  4600|
|      Robert|  4100|
|       Maria|  3000|
+------------+------+

+------------+----------+------+
|EmployeeName|Department|Salary|
+------------+----------+------+
|     Michael|     Sales|  4600|
|      Robert|     Sales|  4100|
+------------+----------+------+

+----------+-----------+
|Department|avg(Salary)|
+----------+-----------+
|     Sales|     3900.0|
|   Finance|     3000.0|
+----------+-----------+



In [38]:
df.withColumn('SalaryBonus', df['Salary'] * 0.1)
df.withColumn('TotalCompensation', df['Salary'] + df['SalaryBonus'])

df.show()

+------------+----------+------+-----------+-----------------+
|EmployeeName|Department|Salary|SalaryBonus|TotalCompensation|
+------------+----------+------+-----------+-----------------+
|       James|     Sales|  3000|      300.0|           3300.0|
|     Michael|     Sales|  4600|      460.0|           5060.0|
|      Robert|     Sales|  4100|      410.0|           4510.0|
|       Maria|   Finance|  3000|      300.0|           3300.0|
+------------+----------+------+-----------+-----------------+



In [31]:
# Contoh menggunakan window functions
from pyspark.sql.window import Window
from pyspark.sql import functions as F

windowSpec = Window.partitionBy('Department').orderBy('Salary')
df.withColumn('Rank', F.rank().over(windowSpec)).show()

+------------+----------+------+----+
|EmployeeName|Department|Salary|Rank|
+------------+----------+------+----+
|       Maria|   Finance|  3000|   1|
|       James|     Sales|  3000|   1|
|      Robert|     Sales|  4100|   2|
|     Michael|     Sales|  4600|   3|
+------------+----------+------+----+



In [84]:
data = [("The Witcher 3", "RPG", 360000),
        ("Elden Ring", "RPG", 599000),
        ("Persona 3: Reload", "RPG", 779000),
        ("Clair Obscur: Expedition 33", "Turn-based", 499000),
        ("Honkai Star Rail", "Turn-based", 0),
        ("Hollow Knight: Silksong", "Metroidvania", 166000)
        ]

columns = ["name", "genre", "price"]

df_game = spark.createDataFrame(data, columns)

df_game.show()

df_game.printSchema()

+--------------------+------------+------+
|                name|       genre| price|
+--------------------+------------+------+
|       The Witcher 3|         RPG|360000|
|          Elden Ring|         RPG|599000|
|   Persona 3: Reload|         RPG|779000|
|Clair Obscur: Exp...|  Turn-based|499000|
|    Honkai Star Rail|  Turn-based|     0|
|Hollow Knight: Si...|Metroidvania|166000|
+--------------------+------------+------+

root
 |-- name: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- price: long (nullable = true)



In [85]:
from pyspark.sql.functions import col, avg, min, max

# Filtering game mahal
df_expensive = df_game.filter(col("price") > 500000)
df_expensive.show()

# Agregasi: rata-rata, min, max
df_avg_price = df_game.groupBy("genre").agg(avg("price").alias("average_price"))
df_avg_price.show()

df_min_price = df_game.groupBy("genre").agg(min("price").alias("smallest_price"))
df_min_price.show()

df_max_price = df_game.groupBy("genre").agg(max("price").alias("highest_price"))
df_max_price.show()

+-----------------+-----+------+
|             name|genre| price|
+-----------------+-----+------+
|       Elden Ring|  RPG|599000|
|Persona 3: Reload|  RPG|779000|
+-----------------+-----+------+

+------------+-----------------+
|       genre|    average_price|
+------------+-----------------+
|         RPG|579333.3333333334|
|Metroidvania|         166000.0|
|  Turn-based|         249500.0|
+------------+-----------------+

+------------+--------------+
|       genre|smallest_price|
+------------+--------------+
|         RPG|        360000|
|Metroidvania|        166000|
|  Turn-based|             0|
+------------+--------------+

+------------+-------------+
|       genre|highest_price|
+------------+-------------+
|         RPG|       779000|
|Metroidvania|       166000|
|  Turn-based|       499000|
+------------+-------------+



In [88]:
# DF Rating
data_rating = [("The Witcher 3", 9.5),
              ("Elden Ring", 9.8),
              ("Persona 3: Reload", 8.5),
              ("Clair Obscur: Expedition 33", 9.7),
              ("Hollow Knight: Silksong", 8.9),
              ("Red Dead Redemption 2",  9.7)]

columns_rating = ["name", "rating"]

df_rating = spark.createDataFrame(data_rating, columns_rating)

# Inner join
df_join = df_game.join(df_rating, on="name", how="inner")
df_join.show()



# Diskon 40%
df_diskon = df_game.withColumn("discount", col("price") * 0.6)
df_diskon.show()

+--------------------+------------+------+------+
|                name|       genre| price|rating|
+--------------------+------------+------+------+
|Clair Obscur: Exp...|  Turn-based|499000|   9.7|
|          Elden Ring|         RPG|599000|   9.8|
|Hollow Knight: Si...|Metroidvania|166000|   8.9|
|   Persona 3: Reload|         RPG|779000|   8.5|
|       The Witcher 3|         RPG|360000|   9.5|
+--------------------+------------+------+------+

+--------------------+------------+------+--------+
|                name|       genre| price|discount|
+--------------------+------------+------+--------+
|       The Witcher 3|         RPG|360000|216000.0|
|          Elden Ring|         RPG|599000|359400.0|
|   Persona 3: Reload|         RPG|779000|467400.0|
|Clair Obscur: Exp...|  Turn-based|499000|299400.0|
|    Honkai Star Rail|  Turn-based|     0|     0.0|
|Hollow Knight: Si...|Metroidvania|166000| 99600.0|
+--------------------+------------+------+--------+



In [83]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank

windowSpec = Window.partitionBy("genre").orderBy(col("price").desc())

df_peringkat = df_game.withColumn("peringkat_genre", rank().over(windowSpec))
df_peringkat.show()

+--------------------+------------+------+---------------+
|                name|       genre| price|peringkat_genre|
+--------------------+------------+------+---------------+
|Hollow Knight: Si...|Metroidvania|166000|              1|
|   Persona 3: Reload|         RPG|779000|              1|
|          Elden Ring|         RPG|599000|              2|
|       The Witcher 3|         RPG|360000|              3|
|Clair Obscur: Exp...|  Turn-based|499000|              1|
|    Honkai Star Rail|  Turn-based|     0|              2|
+--------------------+------------+------+---------------+



In [89]:
!pip install kaggle

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [98]:
!kaggle datasets download -d kanchana1990/perfume-e-commerce-dataset-2024
!unzip perfume-e-commerce-dataset-2024.zip

Dataset URL: https://www.kaggle.com/datasets/kanchana1990/perfume-e-commerce-dataset-2024
License(s): ODC Attribution License (ODC-By)
  0% 0.00/106k [00:00<?, ?B/s]
100% 106k/106k [00:00<00:00, 245MB/s]
Archive:  perfume-e-commerce-dataset-2024.zip
  inflating: ebay_mens_perfume.csv   
  inflating: ebay_womens_perfume.csv  


In [99]:
df = spark.read.csv("ebay_mens_perfume.csv", header=True, inferSchema=True)
df.printSchema()
df.show(5)

root
 |-- brand: string (nullable = true)
 |-- title: string (nullable = true)
 |-- type: string (nullable = true)
 |-- price: string (nullable = true)
 |-- priceWithCurrency: string (nullable = true)
 |-- available: string (nullable = true)
 |-- availableText: string (nullable = true)
 |-- sold: string (nullable = true)
 |-- lastUpdated: string (nullable = true)
 |-- itemLocation: string (nullable = true)

+--------------+--------------------+---------------+------+-----------------+---------+--------------------+----+--------------------+--------------------+
|         brand|               title|           type| price|priceWithCurrency|available|       availableText|sold|         lastUpdated|        itemLocation|
+--------------+--------------------+---------------+------+-----------------+---------+--------------------+----+--------------------+--------------------+
|          Dior|Christian Dior Sa...|  Eau de Parfum| 84.99|     US $84.99/ea|       10|More than 10 avai...| 116|May 

In [104]:
from pyspark.sql.functions import col, avg, min, max

# Filtering parfum mahal
df_expensive = df.filter(col("price") > 50)
df_expensive.show(10)

# Agregasi: rata-rata, min, max
df_avg_price = df.groupBy("type").agg(avg("price").alias("average_price"))
df_avg_price.show(10)

df_min_price = df.groupBy("type").agg(min("price").alias("smallest_price"))
df_min_price.show(10)

df_max_price = df.groupBy("type").agg(max("price").alias("highest_price"))
df_max_price.show(10)

+------------+--------------------+---------------+------+-----------------+---------+--------------------+----+--------------------+--------------------+
|       brand|               title|           type| price|priceWithCurrency|available|       availableText|sold|         lastUpdated|        itemLocation|
+------------+--------------------+---------------+------+-----------------+---------+--------------------+----+--------------------+--------------------+
|        Dior|Christian Dior Sa...|  Eau de Parfum| 84.99|     US $84.99/ea|       10|More than 10 avai...| 116|May 24, 2024 10:0...|Allen Park, Michi...|
|     AS SHOW|A-v-entus Eau de ...|  Eau de Parfum|109.99|       US $109.99|        8|8 available / 48 ...|  48|May 23, 2024 23:0...|Atlanta, Georgia,...|
|   Unbranded|HOGO BOSS cologne...|Eau de Toilette|   100|       US $100.00|       10|More than 10 avai...|  27|May 22, 2024 21:5...|Dearborn, Michiga...|
|   Unbranded|Parfums De-Marly-...|  Eau de Parfum|    85|        US $

In [105]:
# Diskon 50%
df_diskon = df.withColumn("discount", col("price") * 0.5)
df_diskon.show(10)

+---------------+--------------------+---------------+------+-----------------+---------+--------------------+----+--------------------+--------------------+--------+
|          brand|               title|           type| price|priceWithCurrency|available|       availableText|sold|         lastUpdated|        itemLocation|discount|
+---------------+--------------------+---------------+------+-----------------+---------+--------------------+----+--------------------+--------------------+--------+
|           Dior|Christian Dior Sa...|  Eau de Parfum| 84.99|     US $84.99/ea|       10|More than 10 avai...| 116|May 24, 2024 10:0...|Allen Park, Michi...|  42.495|
|        AS SHOW|A-v-entus Eau de ...|  Eau de Parfum|109.99|       US $109.99|        8|8 available / 48 ...|  48|May 23, 2024 23:0...|Atlanta, Georgia,...|  54.995|
|      Unbranded|HOGO BOSS cologne...|Eau de Toilette|   100|       US $100.00|       10|More than 10 avai...|  27|May 22, 2024 21:5...|Dearborn, Michiga...|    50.0

In [108]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank

windowSpec = Window.partitionBy("type").orderBy(col("price").desc())

df_peringkat = df.withColumn("rankings", rank().over(windowSpec))
df_peringkat.show(10)

+-------------------+--------------------+----------+-----+-----------------+---------+--------------------+--------+--------------------+--------------------+--------+
|              brand|               title|      type|price|priceWithCurrency|available|       availableText|    sold|         lastUpdated|        itemLocation|rankings|
+-------------------+--------------------+----------+-----+-----------------+---------+--------------------+--------+--------------------+--------------------+--------+
|            AS SHOW|Bleu De Eau de pa...|      NULL|89.99|        US $89.99|        5|5 available / 5 sold|       5|May 14, 2024 20:4...|New York,United S...|       1|
|       Calvin Klein|Ck One by Calvin ...|      NULL|23.89|     US $23.89/ea|     NULL|Limited quantity ...|   54052|May 24, 2024 03:5...|Hackensack, New J...|       2|
|          Unbranded|Hawas for him Eau...|         /|15.89|        US $15.89|       10|10 available / 17...|      17|May 24, 2024 09:0...|     shanghai, Ch